In [ ]:
import base64
import requests
import json
from datetime import datetime
import pytz
import requests
import json
import pandas as pd
import numpy as np
import json
from google.colab import files

In [ ]:
pip install --upgrade firebase-admin

In [ ]:
#list of dictionaries containing the secret id as the key and client secret as the value pair
id_secre_list = [{"Key":"value"}]


for i in id_secre_list:
  client_id = list(i.keys())[0]
  client_secret = i[client_id]
  print(client_id)
  print(client_secret)

In [ ]:
encodedData = base64.b64encode(bytes(f"{client_id}:{client_secret}", "ISO-8859-1")).decode("ascii")
authorization_header_string = f"Authorization: Basic {encodedData}"
print(encodedData)

In [ ]:
import requests
import json


def access_token():
  url = "https://marketplace.walmartapis.com/v3/token?grant_type=client_credentials"

  payload='grant_type=client_credentials'
  headers = {
    'grant_type': 'client_credentials',
    'WM_SVC.NAME': 'Walmart Marketplace',
    'WM_QOS.CORRELATION_ID': 'lalalatest123',
    'Authorization': f"Basic {encodedData}",
    #'Content-Type': 'application/json',
    "Content-Type": "application/x-www-form-urlencoded",
    'Accept': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)
  access_token = response.json()["access_token"]
  print(access_token)
  return access_token

In [ ]:
#access_token = access_token()

payload={}
headers = {
  'Authorization': f"Basic {encodedData}",
  'WM_SEC.ACCESS_TOKEN': f"{access_token()}",
  'WM_QOS.CORRELATION_ID': 'lala123ytest',
  'WM_SVC.NAME': 'Walmart Marketplace',
  #'Content-Type': 'application/json',
  "Content-Type": "application/x-www-form-urlencoded",
  'Accept': 'application/json'
}

In [ ]:
wfs_fulfilled_url = "https://marketplace.walmartapis.com/v3/orders?createdStartDate=2022-09-04&limit=200&productInfo=true&shipNodeType=WFSFulfilled"  #WFS fulfilled
seller_fulfilled_url = "https://marketplace.walmartapis.com/v3/orders?createdStartDate=2022-09-04&limit=200&productInfo=true&shipNodeType=SellerFulfilled" #seller fulfilled
threepl_fulfilledurl_url = "https://marketplace.walmartapis.com/v3/orders?createdStartDate=2022-09-04&limit=200&productInfo=true&shipNodeType=3PLFulfilled" #3PLFulfilled

wfs_response = requests.request("GET", wfs_fulfilled_url, headers=headers, data=payload)
seller_response = requests.request("GET", seller_fulfilled_url, headers=headers, data=payload)
threepl_response = requests.request("GET", threepl_fulfilledurl_url, headers=headers, data=payload)


wfs_response = wfs_response.json()
seller_response = seller_response.json()
threepl_response = threepl_response.json()


wfs_filtered_response = wfs_response['list']['elements']['order']
seller_filtered_response = seller_response['list']['elements']['order']
threepl_filtered_response = threepl_response['list']['elements']['order']

filtered_response = wfs_filtered_response + seller_filtered_response + threepl_filtered_response

In [ ]:
print(len(filtered_response))
print(filtered_response)

In [ ]:
#google colab set up 

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

#firebase_admin.delete_app(firebase_admin.get_app())  #incase firebase has already bee n installed, given issues then uncomment this line 


cred = credentials.Certificate('/content/firestore email-automation-pt2.json')
app = firebase_admin.initialize_app(cred)
db = firestore.client()

In [ ]:
from pytz import timezone
import datetime
from datetime import datetime


def convert_to_est(time):
  dtc_utc = datetime.utcfromtimestamp(time / 1000.0)

  #create an estern time zone object
  eastern_time_zone = pytz.timezone('US/Eastern')

  #convert the datetime object to Eastern time zone
  eastern_time = dtc_utc.replace(tzinfo=pytz.UTC).astimezone(eastern_time_zone)

  return eastern_time

In [ ]:
#write data to firestore

import pandas as pd

table = pd.DataFrame([])


for i in filtered_response:
  i['order_date_est'] = convert_to_est(i['orderDate']).strftime("%m-%d-%Y")
  
  products_onthis_order = len(i['orderLines']['orderLine'])
  i['products_onthis_order'] = products_onthis_order

  item_date_delivered = np.unique([(np.int64(i['orderLines']['orderLine'][k]['statusDate'])) for k in range(products_onthis_order)])
  item_date_delivered_est = [convert_to_est(k) for k in item_date_delivered][0].strftime("%m-%d-%Y")
  i['Delivered_date_est'] = item_date_delivered_est #the date the item was actually delivered. If item is shipped then it is the date the item should been delivered by

  item_status = [(i['orderLines']['orderLine'][k]['orderLineStatuses']['orderLineStatus'][0]['status']) for k in range(products_onthis_order)]
  
  if all(i == "Delivered" for i in item_status):
    i['order_status'] = "All order line items are delivered"
  else:
     i['order_status'] = "NOT every item Delivered"
 
 
  users_ref = db.collection(U'customer_order_ids').document(i['customerOrderId'])
  doc = users_ref.get().to_dict()
  
  if doc != None:
    if ("message_1_sent" not in doc) or ("review_message_sent" not in doc):
      i['review_message_sent'] = "No Messages Sent"
    # elif ("message_1_sent" in doc) and (doc['message_1_sent'] == "No Messages Sent"):
    #   i['review_message_sent'] = "No Messages Sent"
    #   i['subject_line'] = "{first_name}, Thank you for your order"
    # elif ("review_message_sent" in doc) and (doc['review_message_sent'] == "No Messages Sent"):
    #   i['review_message_sent'] = "No Messages Sent"
    #   i['subject_line'] = "{first_name}, Thank you for your order"
  else: #document is empty/doesn't exist
    print(doc) 
    doc_ref = db.collection(U'customer_order_ids').document(i['customerOrderId'])
    doc_ref.set(i)
    #pass
  



In [ ]:
pip install git+https://github.com/sendinblue/APIv3-python-library.git

In [ ]:
#send emails function

from __future__ import print_function
import sib_api_v3_sdk
from sib_api_v3_sdk.rest import ApiException

configuration = sib_api_v3_sdk.Configuration()
configuration.api_key['api-key'] = 'API KEY HERE'
api_instance = sib_api_v3_sdk.TransactionalEmailsApi(sib_api_v3_sdk.ApiClient(configuration))

def send_email(from_name, from_email, to_email, customer_name, subject, body):


  sender = {"name":f"{from_name}","email": f"{from_email}"}
  replyTo = {"name":f"{from_name}", "email": f"{from_email}"}
  subject = subject
  html_content = f"<html><body><h1> {body} </h1></body></html>"
  to = [{"email":f"{to_email}"}]
  send_smtp_email = sib_api_v3_sdk.SendSmtpEmail(to=to, reply_to=replyTo, html_content=html_content, sender=sender, subject=subject, params={"FIRSTNAME": f"{customer_name}", "LASTNAME": "<p>Here's an image:</p><img src='https://i5.walmartimages.com/asr/2aa97818-4b49-466f-a0a2-7eabe7d6f6c8.66277b9b129d8652d5f1d55f8d90e30f.jpeg?odnHeight=104&odnWidth=104&odnBg=FFFFFF'>"})

  try:
      api_response = api_instance.send_transac_email(send_smtp_email)
      print(api_response)
  except ApiException as e:
      print("Exception when calling SMTPApi->send_transac_email: %s\n" % e)

In [ ]:
#function to determine the right review link for any given product

image_url = []
def review_url(sku):

  url  = f"https://marketplace.walmartapis.com/v3/items/{sku}"

  payload={}
  headers = {
    'Authorization': f"Basic {encodedData}",
    'WM_SEC.ACCESS_TOKEN': f"{access_token()}",
    'WM_QOS.CORRELATION_ID': 'lalatyest1321',
    'WM_SVC.NAME': 'Walmart Marketplace',
    #'Content-Type': 'application/json',
    "Content-Type": "application/x-www-form-urlencoded",
    'Accept': 'application/json'
  }


   #let's look for the GTIN based on the product SKU
  item_gtin_response = requests.request("GET", url, headers=headers, data=payload)
  print(item_gtin_response)
  item_gtin_response_filtered = item_gtin_response.json()['ItemResponse'][0]['gtin'] #['items'][0]['itemId']


  #now let's find the product ID from the GTIN
  url = f"https://marketplace.walmartapis.com/v3/items/walmart/search?gtin={item_gtin_response_filtered}"
  item_id_response = requests.request("GET", url, headers=headers, data=payload)
  
  #global image_url 
  image_url.append(item_id_response.json()['items'][0]['images'][0]['url'])
  item_id_response = item_id_response.json()['items'][0]['itemId']
  

  #finally - the review url
  review_url = f"https://www.walmart.com/reviews/write-review?productId={item_id_response}"
  
  return review_url


In [ ]:
  #email format template in HTML
  
  def review_email_template(customer_name, order_id_url, order_id, first_item_name, item_review_url, image_url=image_url):
    return """ 
    <!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
    <html xmlns="http://www.w3.org/1999/xhtml" xmlns:o="urn:schemas-microsoft-com:office:office">

    <head>
        <meta charset="UTF-8">
        <meta content="width=device-width, initial-scale=1" name="viewport">
        <meta name="x-apple-disable-message-reformatting">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">
        <meta content="telephone=no" name="format-detection">
        <title></title>
        <!--[if (mso 16)]>
        <style type="text/css">
        a {text-decoration: none;}
        </style>
        <![endif]-->
        <!--[if gte mso 9]><style>sup { font-size: 100% !important; }</style><![endif]-->
        <!--[if gte mso 9]>
    <xml>
        <o:OfficeDocumentSettings>
        <o:AllowPNG></o:AllowPNG>
        <o:PixelsPerInch>96</o:PixelsPerInch>
        </o:OfficeDocumentSettings>
    </xml>
    <![endif]-->
    </head>

    <body>
        <div class="es-wrapper-color">
            <!--[if gte mso 9]>
          <v:background xmlns:v="urn:schemas-microsoft-com:vml" fill="t">
            <v:fill type="tile" color="#ffffff"></v:fill>
          </v:background>
        <![endif]-->
            <table class="es-wrapper" width="100%" cellspacing="0" cellpadding="0">
                <tbody>
                    <tr>
                        <td class="esd-email-paddings" valign="top">
                            <table cellpadding="0" cellspacing="0" class="esd-header-popover es-header" align="center">
                                <tbody>
                                    <tr>
                                        <td class="esd-stripe" align="center">
                                            <table bgcolor="#ffffff" class="es-header-body" align="center" cellpadding="0" cellspacing="0" width="600">
                                                <tbody>
                                                    <tr>
                                                        <td class="esd-structure esdev-adapt-off es-p20" align="left">
                                                            <table cellpadding="0" cellspacing="0" width="100%">
                                                                <tbody>
                                                                    <tr>
                                                                        <td width="560" class="es-m-p0r esd-container-frame" valign="top" align="center">
                                                                        </td>
                                                                    </tr>
                                                                </tbody>
                                                            </table>
                                                        </td>
                                                    </tr>
                                                    <tr>
                                                        <td class="esd-structure es-p20t es-p20r es-p20l" align="left">
                                                            <table cellpadding="0" cellspacing="0" width="100%">
                                                                <tbody>
                                                                    <tr>
                                                                        <td width="560" class="esd-container-frame" align="center" valign="top">
                                                                            <table cellpadding="0" cellspacing="0" width="100%">
                                                                                <tbody>
                                                                                    <tr>
                                                                                        <td align="center" class="esd-block-text">
                                                                                            <h1 style="font-size: 24px;"> """ + customer_name + """, Thank you for your order</h1>
                                                                                        </td>
                                                                                    </tr>
                                                                                    <tr>
                                                                                        <td align="center" class="esd-block-spacer es-p20" style="font-size:0">
                                                                                            <table border="0" width="100%" height="100%" cellpadding="0" cellspacing="0">
                                                                                                <tbody>
                                                                                                    <tr>
                                                                                                        <td style="border-bottom: 0px solid #cccccc; background: unset; height: 10px; width: 100%; margin: 0px;"></td>
                                                                                                    </tr>
                                                                                                </tbody>
                                                                                            </table>
                                                                                        </td>
                                                                                    </tr>
                                                                                    <tr>
                                                                                        <td align="left" class="esd-block-text es-p5t es-p5b">
                                                                                            <p style="font-size: 16px; line-height: 1.5; margin: 0 0 20px; font-weight: 500">We would love to hear your thoughts on the product you purchased on Walmart.com.<br><br>Your honest feedback can help us improve our products and services to better meet your needs, and it can also assist other shoppers in making informed decisions.<br><br>To leave a review, simply click on the link below, it only takes a few minutes, and it means the world to us.</p>
                                                                                        </td>
                                                                                    </tr>
                                                                                </tbody>
                                                                            </table>
                                                                        </td>
                                                                    </tr>
                                                                </tbody>
                                                            </table>
                                                        </td>
                                                    </tr>
                                                </tbody>
                                            </table>
                                        </td>
                                    </tr>
                                </tbody>
                            </table>
                            <table cellpadding="0" cellspacing="0" class="es-content esd-footer-popover" align="center">
                                <tbody>
                                    <tr>
                                        <td class="esd-stripe" align="center">
                                            <table bgcolor="#ffffff" class="es-content-body" align="center" cellpadding="0" cellspacing="0" width="600">
                                                <tbody>
                                                    <tr>
                                                        <td class="esd-structure es-p20t es-p20r es-p20l" align="left">
                                                            <table cellpadding="0" cellspacing="0" width="100%">
                                                                <tbody>
                                                                    <tr>
                                                                        <td width="560" class="esd-container-frame" align="left" valign="top">
                                                                            <table cellpadding="0" cellspacing="0" width="100%">
                                                                                <tbody>
                                                                                    <tr>
                                                                                        <td align="left" class="esd-block-text es-m-txt-c es-p20t">
                                                                                            <p style="color: #000000; font-size: 24px">Order ID: <a href='""" + order_id_url + """'>""" + order_id + """</a></p>
                                                                                        </td>
                                                                                    </tr>
                                                                                    <tr>
                                                                                        <td align="center" class="esd-block-spacer es-p5t es-p5b" style="font-size:0">
                                                                                            <table border="0" width="100%" height="100%" cellpadding="0" cellspacing="0">
                                                                                                <tbody>
                                                                                                    <tr>
                                                                                                        <td style="border-bottom: 1px solid #a0937d; background: none; height: 1px; width: 100%; margin: 0px;"></td>
                                                                                                    </tr>
                                                                                                </tbody>
                                                                                            </table>
                                                                                        </td>
                                                                                    </tr>
                                                                                </tbody>
                                                                            </table>
                                                                        </td>
                                                                    </tr>
                                                                </tbody>
                                                            </table>
                                                        </td>
                                                    </tr>
                                                    <tr>
                                                        <td class="esd-structure esdev-adapt-off es-p20" align="left" esd-custom-block-id="731032" esdev-config="h100">
                                                            <table width="560" cellpadding="0" cellspacing="0" class="esdev-mso-table">
                                                                <tbody>
                                                                    <tr>
                                                                        <td class="esdev-mso-td" valign="top">
                                                                            <table cellpadding="0" cellspacing="0" class="es-left" align="left">
                                                                                <tbody>
                                                                                    <tr>
                                                                                        <td width="219" class="esd-container-frame" align="left">
                                                                                            <table cellpadding="0" cellspacing="0" width="100%">
                                                                                                <tbody>
                                                                                                    <tr>
                                                                                                        <td align="center" class="esd-block-image" style="font-size: 0px;"><a target="_blank" href=https://viewstripo.email"><img class="adapt-img p_image" src='""" + image_url[0] + """'alt="Marshall Monitor" style="display: block;" width="125" title="Marshall Monitor"></a></td>
                                                                                                    </tr>
                                                                                                </tbody>
                                                                                            </table>
                                                                                        </td>
                                                                                    </tr>
                                                                                </tbody>
                                                                            </table>
                                                                        </td>
                                                                        <td width="20"></td>
                                                                        <td class="esdev-mso-td" valign="top">
                                                                            <table cellpadding="0" cellspacing="0" class="es-left" align="left">
                                                                                <tbody>
                                                                                    <tr>
                                                                                        <td width="125" align="left" class="esd-container-frame">
                                                                                            <table cellpadding="0" cellspacing="0" width="100%">
                                                                                                <tbody>
                                                                                                    <tr>
                                                                                                        <td align="center" class="esd-block-spacer es-p10" style="font-size:0">
                                                                                                            <table border="0" width="100%" height="100%" cellpadding="0" cellspacing="0">
                                                                                                                <tbody>
                                                                                                                    <tr>
                                                                                                                        <td style="border-bottom: 0px solid #cccccc; background: unset; height: 20px; width: 100%; margin: 0px;"></td>
                                                                                                                    </tr>
                                                                                                                </tbody>
                                                                                                            </table>
                                                                                                        </td>
                                                                                                    </tr>
                                                                                                    <tr>
                                                                                                        <td align="left" class="esd-block-text es-p20t es-p20b es-m-p0t es-m-p0b es-m-txt-l">
                                                                                                            <h3 style="font-size: 14px;"><strong class="p_name">"""+ first_item_name +"""</strong></h3>
                                                                                                        </td>
                                                                                                    </tr>
                                                                                                </tbody>
                                                                                            </table>
                                                                                        </td>
                                                                                    </tr>
                                                                                </tbody>
                                                                            </table>
                                                                        </td>
                                                                        <td width="20"></td>
                                                                        <td class="esdev-mso-td" valign="top">
                                                                            <table cellpadding="0" cellspacing="0" class="es-right" align="right">
                                                                                <tbody>
                                                                                    <tr>
                                                                                        <td width="176" align="left" class="esd-container-frame">
                                                                                            <table cellpadding="0" cellspacing="0" width="100%">
                                                                                                <tbody>
                                                                                                    <tr>
                                                                                                        <td align="center" class="esd-block-spacer es-p15" style="font-size:0">
                                                                                                            <table border="0" width="100%" height="100%" cellpadding="0" cellspacing="0">
                                                                                                                <tbody>
                                                                                                                    <tr>
                                                                                                                        <td style="border-bottom: 0px solid #cccccc; background: unset; height: 30px; width: 100%; margin: 0px;"></td>
                                                                                                                    </tr>
                                                                                                                </tbody>
                                                                                                            </table>
                                                                                                        </td>
                                                                                                    </tr>
                                                                                                </tbody>
                                                                                            </table>
                                                                                        </td>
                                                                                    </tr>
                                                                                    <tr>
                                                                                        <td width="176" align="left" class="esd-container-frame">
                                                                                            <table cellpadding="0" cellspacing="0" width="100%">
                                                                                                <tbody>
                                                                                                    <tr>
                                                                                                        <td align="center" class="esd-block-button"><span class="es-button-border" style="background: #3d85c6;"><a href='"""+ item_review_url + """'style="display: inline-block; padding: 10px 20px; background-color: #007bff; color: #ffffff; text-decoration: none; border-radius: 5px; font-size: 16px; line-height: 1.5;">Leave a Review</a></span></td> 
                                                                                                    </tr>
                                                                                                </tbody>
                                                                                            </table>
                                                                                        </td>
                                                                                    </tr>
                                                                                </tbody>
                                                                            </table>
                                                                        </td>
                                                                    </tr>
                                                                </tbody>
                                                            </table>
                                                        </td>
                                                    </tr>
                                                    <tr>
                                                        <td class="esd-structure es-p20t es-p20r es-p20l" align="left">
                                                            <table cellpadding="0" cellspacing="0" width="100%">
                                                                <tbody>
                                                                    <tr>
                                                                        <td width="560" class="esd-container-frame" align="center" valign="top">
                                                                            <table cellpadding="0" cellspacing="0" width="100%">
                                                                                <tbody>
                                                                                    <tr>
                                                                                        <td align="center" class="esd-block-spacer es-p20" style="font-size:0">
                                                                                            <table border="0" width="100%" height="100%" cellpadding="0" cellspacing="0">
                                                                                                <tbody>
                                                                                                    <tr>
                                                                                                        <td style="border-bottom: 0px solid #cccccc; background: unset; height: 20px; width: 100%; margin: 0px;"></td>
                                                                                                    </tr>
                                                                                                </tbody>
                                                                                            </table>
                                                                                        </td>
                                                                                    </tr>
                                                                                </tbody>
                                                                            </table>
                                                                        </td>
                                                                    </tr>
                                                                </tbody>
                                                            </table>
                                                        </td>
                                                    </tr>
                                                    <tr>
                                                </tbody>
                                            </table>
                                        </td>
                                    </tr>
                                </tbody>
                            </table>
                        </td>
                    </tr>
                </tbody>
            </table>
        </div>
    </body>

    </html>
    """

In [ ]:
from pytz import timezone
import datetime

est_timezone = timezone('EST')
current_time = datetime.datetime.now(est_timezone)
how_many_days_ago = datetime.timedelta(8)
cutoff_date = (current_time - how_many_days_ago)  

In [ ]:
#retrieve data from firebase of those who's not been delivered & send send emails

import pandas as pd
import numpy as np
import json
from google.colab import files
from pytz import timezone
import datetime
from datetime import datetime


def convert_to_est(time):
  dtc_utc = datetime.utcfromtimestamp(time / 1000.0)

  #create an estern time zone object
  eastern_time_zone = pytz.timezone('US/Eastern')

  #convert the datetime object to Eastern time zone
  eastern_time = dtc_utc.replace(tzinfo=pytz.UTC).astimezone(eastern_time_zone)

  return eastern_time


users_ref = db.collection(U'customer_order_ids')
docs = users_ref.get()

for doc in docs:
  data = doc.to_dict()
  print(data)  
  print("\n\n-----\n\n")
  
  products_onthis_order = len(data['orderLines']['orderLine'])

  customer_name = data['shippingInfo']['postalAddress']['name'].split()[0]
  customer_order_id =  data['customerOrderId']
  purchase_order_id =  data['purchaseOrderId']
  customer_email_id =  data['customerEmailId']
  order_date =  np.int64(data['orderDate'])
  order_date_est = convert_to_est(order_date)
  customer_email = data['fake_email']
    

  #get the unique item attributes, if we have 3 of the same SKU - we only care that the SKU exists atleast once within the order
  item_name = np.unique([(data['orderLines']['orderLine'][i]['item']['productName']) for i in range(products_onthis_order)])
  item_sku = np.unique([(data['orderLines']['orderLine'][i]['item']['sku']) for i in range(products_onthis_order)])
  item_date_delivered = np.unique([(np.int64(data['orderLines']['orderLine'][i]['statusDate'])) for i in range(products_onthis_order)])
  item_date_delivered_est = [convert_to_est(i) for i in item_date_delivered ][0]
  item_status = [(data['orderLines']['orderLine'][i]['orderLineStatuses']['orderLineStatus'][0]['status']) for i in range(products_onthis_order)]
  item_refunded = [data['orderLines']['orderLine'][i]['refund'] for i in range(products_onthis_order)]

  
  #item_date_delivered_est = item_date_delivered_est.pytz.timezone('EST')

  
  if "review_message_sent" in data:
    key_exists = True
  else:
    key_exists = False

  def send_message():
      subject = f"{customer_name}, Thank you for your order"
      order_id_url = f"https://www.walmart.com/orders/{customer_order_id}"
      send_email(from_name = "EXTRASHADE", from_email = "yumeldavid@gmail.com", to_email = "yumeldavid@gmail.com", customer_name = customer_name, subject=subject, body=review_email_template(customer_name, order_id_url, customer_order_id, item_name[0][:40], review_url(item_sku[0])))
      
      
      data['review_message_sent'] = "We would love to hear your thoughts on the product you purchased on Walmart.com.Your honest feedback can help us improve our products and services to better meet your needs, and it can also assist other shoppers in making informed decisions. To leave a review, simply click on the link below, it only takes a few minutes, and it means the world to us."
      data['review_message_sent_date'] = datetime.now(pytz.timezone('EST'))
      data['review_message_sent_date_est'] = datetime.now(pytz.timezone('EST')).strftime('%m-%d-%Y')
      data['subject_line'] = "{first_name}, Thank you for your order"
      doc_ref = db.collection(U'customer_order_ids').document(data['customerOrderId'])
      doc_ref.set(data)
      print("email sent")


  send_message()

  if all(i == "Delivered" for i in item_status) and all(i == None for i in item_refunded) and (item_date_delivered_est <= cutoff_date):
    if ("review_message_sent" in data) and (data['review_message_sent'] == "No Messages Sent"):
      send_message()
    elif ("message_1_sent" in data) and (data['message_1_sent'] == "No Messages Sent"):
      send_message()
     
  else:
    print("NOT every item Delivered")
    

In [ ]:
#getting all email sent in sendblue to then summarize stats

from __future__ import print_function
import time
import sib_api_v3_sdk
from sib_api_v3_sdk.rest import ApiException
from pprint import pprint

# Configure API key authorization: api-key
configuration = sib_api_v3_sdk.Configuration()
configuration.api_key['api-key'] = 'API KEY'


# create an instance of the API class
api_instance = sib_api_v3_sdk.TransactionalEmailsApi(sib_api_v3_sdk.ApiClient(configuration))
limit = 2500 # int | Number limitation for the result returned (optional) (default to 2500)
offset = 0 # int | Beginning point in the list to retrieve from. (optional) (default to 0)
start_date = '2023-01-25' # str | Mandatory if endDate is used. Starting date of the report (YYYY-MM-DD). Must be lower than equal to endDate (optional)
end_date = '2023-04-16' # str | Mandatory if startDate is used. Ending date of the report (YYYY-MM-DD). Must be greater than equal to startDate (optional)
sort = 'desc' # str | Sort the results in the ascending/descending order of record creation. Default order is **descending** if `sort` is not passed (optional) (default to desc)
#event = "opened"

try:
    # Get all your transactional email activity (unaggregated events)
    #api_response = api_instance.get_email_event_report(limit=limit, offset=offset, start_date=start_date, end_date=end_date, days=days, email=email, event=event, tags=tags, message_id=message_id, template_id=template_id, sort=sort)
    api_response = api_instance.get_email_event_report(limit=limit, offset=offset, start_date=start_date, end_date=end_date, sort=sort)
    pprint(api_response)
    #print(api_response['events'])
except ApiException as e:
    print("Exception when calling TransactionalEmailsApi->get_email_event_report: %s\n" % e)

In [ ]:
import pandas as pd

response_dict =  api_response.to_dict()['events']
delivered_opened_dict = []

#let's clean our list of dictionaries so that it only includes delivered & opened 
for i in response_dict:
  if (i['event'] == "Opened") or (i['event'] == "delivered"):
    delivered_opened_dict.append(i)

#now let's write it in firebase our database
for i in delivered_opened_dict:
  doc_ref = db.collection(U'email_activity_data').document(i['message_id'])
  doc_ref.set(i)

print(len(delivered_opened_dict))

In [ ]:
#now let's get the email summary per user

#retrieve data from firestore
users_ref = db.collection(U'email_activity_data')
docs = users_ref.get()

retrieved_docs_dict = []

for doc in docs:
  data = doc.to_dict()
  retrieved_docs_dict. append(data)


#now let's get the running stat from all previous email activity and write it to firestore 
df = pd.DataFrame(retrieved_docs_dict)
emails = df['_from'].unique()

stats = []

for i in emails: 

  delivered = len(df[(df['_from']==f"{i}")&(df['event']=='delivered')])
  opened = len(df[(df['_from']==f"{i}")&(df['event']=='delivered')])
  open_rate = (delivered/opened)*100
  stats.append({
      'email': i, 
      'delivered': delivered,
      'opened': opened,
      'open_rate': open_rate   
                })

#write it in firebase
for i in stats:
  doc_ref = db.collection(U'email_activity_running_stat').document(i['email'])
  doc_ref.set(i)


print(stats)
